In [1]:
import os
import csv
import math
import wandb
import numpy as np
from pathlib import Path 

from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from wandb.keras import WandbCallback
from keras.layers import Dense, Flatten, Dropout

Using TensorFlow backend.


In [2]:
#Getting the latest file from the folder
record_path = Path("RecordSummary")
training_data_list = list(filter(os.path.isfile,record_path.glob('*.csv')))
training_data_list.sort(key=lambda x: os.path.getmtime(x))
training_data = training_data_list[-1]

x_train_percent = 60
x_test_percent = 100-x_train_percent
x_train = []
x_test = []
y_train = []
y_test = []

with open(training_data) as csv_file:
    data = list(csv.reader(csv_file))
    lines = list(map(list, zip(*data)))

nr_x_train = math.ceil(len(lines[0])*x_train_percent/100)
lines = np.asarray(lines)


#X stands for input, Y stands for output

x_train = lines[1:, 0:nr_x_train]
y_train = lines[0, 0:nr_x_train]

x_test = lines[1:, nr_x_train:] 
y_test = lines[0, nr_x_train:]
x_train = x_train.transpose()
x_test = x_test.transpose()

x_train = [a.reshape(4,96) for a in x_train]
x_test = [a.reshape(4,96) for a in x_test]

is_up_y_train = y_train=='up'
is_up_y_test = y_test=='up'
labels = ["Not UP", "UP"]

In [3]:
# logging code
run = wandb.init(project="eeg-up-down")
config = run.config


In [4]:
channels = np.shape(x_train)[1]
samples = np.shape(x_train)[2]

print(channels)
print(samples)

4
96


In [5]:
print(np.shape(x_train))
print(np.shape(is_up_y_train))
print(np.shape(x_test))
print(np.shape(is_up_y_test))

(132, 4, 96)
(132,)
(87, 4, 96)
(87,)


In [6]:
print(type(x_train))
print(type(is_up_y_train))

<class 'list'>
<class 'numpy.ndarray'>


In [7]:
#converting <class 'list'> to <class 'numpy.ndarray'>
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)

In [8]:
# create model
model=Sequential()
model.add(Flatten(input_shape=(channels,samples)))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam',
                metrics=['accuracy'])

In [11]:
config.epochs = 1000
# Fit the model
model.fit(x_train, is_up_y_train, epochs=config.epochs, validation_data=(x_test, is_up_y_test),
                    callbacks=[WandbCallback(labels=labels)])

Train on 132 samples, validate on 87 samples
Epoch 1/10
132/132 [==============================] - 1s 10ms/step - loss: 111.3758 - accuracy: 0.0530 - val_loss: 1552.2386 - val_accuracy: 0.0115
Epoch 2/10
132/132 [==============================] - 0s 296us/step - loss: 94.4200 - accuracy: 0.0076 - val_loss: 1584.6151 - val_accuracy: 0.0230
Epoch 3/10
132/132 [==============================] - 0s 303us/step - loss: 80.4011 - accuracy: 0.0455 - val_loss: 1456.4386 - val_accuracy: 0.0345
Epoch 4/10
132/132 [==============================] - 0s 296us/step - loss: 76.7734 - accuracy: 0.0152 - val_loss: 1277.9333 - val_accuracy: 0.0575


C:\Users\jocom\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (1.335608). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)
C:\Users\jocom\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.668293). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


Epoch 5/10
132/132 [==============================] - 0s 311us/step - loss: 80.3732 - accuracy: 0.0227 - val_loss: 1165.1611 - val_accuracy: 0.0460
Epoch 6/10
132/132 [==============================] - 0s 296us/step - loss: 75.2962 - accuracy: 0.0076 - val_loss: 1153.8724 - val_accuracy: 0.0345
Epoch 7/10
132/132 [==============================] - 0s 303us/step - loss: 77.3364 - accuracy: 0.0758 - val_loss: 1033.1155 - val_accuracy: 0.0805
Epoch 8/10
132/132 [==============================] - 0s 296us/step - loss: 71.4041 - accuracy: 0.0227 - val_loss: 1002.6828 - val_accuracy: 0.0805
Epoch 9/10
132/132 [==============================] - 0s 296us/step - loss: 65.4124 - accuracy: 0.0303 - val_loss: 1017.9158 - val_accuracy: 0.0575
Epoch 10/10
132/132 [==============================] - 0s 311us/step - loss: 63.3340 - accuracy: 0.0379 - val_loss: 958.0211 - val_accuracy: 0.0690
